In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("talk")
sns.set_style("darkgrid")

In [6]:
import itertools 

def create_df_from_combinations(input_dict):
    variable_names = []
    product_input_tuple = ()
    for key, value in input_dict.items():
        variable_names.append(key)
        product_input_tuple += (value,) # this is one way to add things to a tuple, apparently

    return pd.DataFrame(list(itertools.product(*product_input_tuple)), columns=variable_names)

Scenarios:
- Archetypes
- Drugs: DP vs 14-day drug (archetype dependent)
- Screening type: IPT vs IST
- Interval: once per term vs once per month
- Background transmission intensity: 5 levels
- Within-school coverage: 5 levels?
- Treatment-seeking rate: low vs default (not combinatorial)
- Bednet coverage: default vs high (not combinatorial)

For "not combinatorial", this means adding a single row for that variation, rather than including all combinations


- Sahel: SPAQ (would also consider ASAQ as an alternative)
- Coastal West Africa: SPAQ (would also consider ASAQ as an alternative)
- Central Africa: ASAQ
- East Africa: ASAQ and SPAQ (the later is only because Sarah mentioned they are trying it for SMC in some areas of Uganda)
- Southern Africa: ASAQ"

In [21]:
def basic_scenarios(archetype):
    if archetype == "Southern" or archetype == "Central":
        drugs = ["ASAQ","DP"]
    elif archetype == "Eastern":
        drugs = ["ASAQ","SPAQ","DP"]
    elif archetype == "Coastal Western" or archetype == "Sahel":
        drugs = ["SPAQ","DP"]
    
    scenario_variations = {
        "drug_type": drugs,
        "screen_type": ["IPT", "IST"],
        "interval": ["term", "month"],
        "school_coverage": [0.6,0.8,1.0],
        "itn_coverage": ["default"],
        "hs_rate": ["default"]
    }

    df_combinatorial = create_df_from_combinations(scenario_variations)

    df_not_combinatorial = pd.DataFrame({
        "drug_type": ["DP", "DP", "DP"],
        "screen_type": ["IPT", "IPT", "IPT"],
        "interval": ["month", "month", "day"],
        "school_coverage": [0.8, 0.8, 1.0],
        "itn_coverage": ["low", "default", "default"],
        "hs_rate": ["default", "high", "default"]
    })

    scenario_df = pd.concat([df_combinatorial, df_not_combinatorial], ignore_index=True)
    scenario_df["scenario_number"] = range(len(scenario_df))
    scenario_df["archetype"] = archetype
    
    return scenario_df

In [22]:
archetype_list = ["Southern", "Central", "Eastern", "Coastal Western", "Sahel"]

In [25]:
df_list = []
for a in archetype_list:
    df_list.append(basic_scenarios(a))
df_full = pd.concat(df_list, ignore_index=True)

In [39]:
# Purely aesthetic: put archetype first
cols = list(df_full.columns.values)
df_full = df_full[[cols[-1]]+cols[:-1]]

In [41]:
df_full.to_csv("scenario_master_list.csv", index=False)